In [1]:
%%bash
pip install pandas-datareader
pip install get-all-tickers

  Created wheel for get-all-tickers: filename=get_all_tickers-1.7-cp36-none-any.whl size=4242 sha256=f8af6b01dd16efeb8e561bfaedb0ccbc1fa2e4be8d0c4d346e3903eb8c3ea9ce
  Stored in directory: /root/.cache/pip/wheels/d0/8b/ef/6cd3580f4b479aef881a32bd937c282982e3d186b617a663ac
Successfully built get-all-tickers


In [2]:

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History
from collections import Iterable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import pandas as pd
import time
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
from get_all_tickers import get_tickers as gt


In [3]:
#getting the labels
news_data = pd.read_csv('Combined_News_DJIA.csv', index_col=0)
labels = news_data[['Label']]
labels.head(5)

,Label
Date,
2008-08-08,0
2008-08-11,1
2008-08-12,0
2008-08-13,0
2008-08-14,1


In [4]:
result = pd.read_csv('result.csv')
result = result.drop(columns=['Unnamed: 0'])
result.set_index('Date', inplace=True)
result.head(5)

,pos_mean,neg_mean,neu_mean,comp_mean
Date,,,,
2008-08-08,0.03728,0.19956,0.76320,-0.354260
2008-08-11,0.06180,0.16404,0.77408,-0.158716
2008-08-12,0.06444,0.14580,0.78980,-0.191800
2008-08-13,0.05620,0.14432,0.79944,-0.141844
2008-08-14,0.08372,0.16136,0.75488,-0.136908


In [5]:
stock_data = pd.read_csv('upload_DJIA_table.csv')
stock_data.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [6]:
# Adjusting data as per date
stock_data.sort_values(by=['Date'], inplace=True, ascending=True)
stock_data

,Date,Open,High,Low,Close,Volume,Adj Close
1988,2008-08-08,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
...,...,...,...,...,...,...,...
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234


# Comparison of VADER generated sentiment and Expertised sentiment

Getting DJIA stock market data

In [7]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL
list_of_tickers=['DJIA'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)

#merge the stock data with the sentiment data (VADER and Expertise) individually

expertise_data = pd.merge(labels, data, right_index=True, left_index=True)

vader_data=pd.merge(result, data, right_index=True, left_index=True)

#we will consider the compund score of the VADER since it has the compound score of the pos, neg and neu

vader_data = vader_data.drop(columns=['comp_mean'])
vader_data

,pos_mean,neg_mean,neu_mean,High,Low,Open,Volume,Adj Close
Date,,,,,,,,
2008-08-08,0.03728,0.19956,0.76320,11808.490234,11344.230469,11432.089844,4966810000,11734.320312
2008-08-11,0.06180,0.16404,0.77408,11933.549805,11580.190430,11729.669922,5067310000,11782.349609
2008-08-12,0.06444,0.14580,0.78980,11830.389648,11541.429688,11781.700195,4711290000,11642.469727
2008-08-13,0.05620,0.14432,0.79944,11689.049805,11377.370117,11632.809570,4787600000,11532.959961
2008-08-14,0.08372,0.16136,0.75488,11744.330078,11399.839844,11532.070312,4064000000,11615.929688
...,...,...,...,...,...,...,...,...
2016-06-27,0.09048,0.15316,0.75636,17355.210938,17005.439453,17355.210938,5431220000,17140.240234
2016-06-28,0.09972,0.13696,0.76328,17453.929688,17174.960938,17190.509766,4385810000,17409.720703
2016-06-29,0.09188,0.18680,0.72124,17737.380859,17456.019531,17456.019531,4241740000,17694.679688


# Generation of lags for the time series data

In [8]:
# Adjusting data as per date
data.sort_values(by=['Date'], inplace=True, ascending=True)

In [9]:
#Scaling the variables(x) and variable(y)
sca_vader = vader_data.iloc[:,:]
sca_expertise = expertise_data.iloc[:,:]

In [11]:
#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_vader=sc.fit_transform(sca_vader)
scaled_vader = pd.DataFrame(data=scaled_vader,columns=['pos_mean','neu_mean','neg_mean','Open','High','Low','Volume','Adj Close'])


sc = StandardScaler()
scaled_exp=sc.fit_transform(sca_expertise)
scaled_exp = pd.DataFrame(data=scaled_exp,columns=['Label','Open','High','Low','Volume','Adj Close'])

Splitting the data for creating training and testing sets

In [14]:
scaled_vader.iloc[:,7]

0      -0.549983
1      -0.534702
2      -0.579205
3      -0.614045
4      -0.587648
          ...   
1984    1.169894
1985    1.255628
1986    1.346287
1987    1.421150
1988    1.427316
Name: Adj Close, Length: 1989, dtype: float64

In [16]:
x_vader = scaled_vader.iloc[:,0:7]
y_vader = scaled_vader.iloc[:,7]

x_exp = scaled_exp.iloc[:,0:5]
y_exp = scaled_exp.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)
x_train_vader, x_test_vader, y_train_vader, y_test_vader = train_test_split(x_vader, y_vader, test_size=0.30,random_state=1)
x_train_exp, x_test_exp, y_train_exp, y_test_exp = train_test_split(x_exp, y_exp, test_size=0.30,random_state=1)

In [17]:
class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [18]:
def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)

In [20]:
TIME_STEPS = 1
# x_train1, y_train1 = dataset(x_train,y_train, y_train, TIME_STEPS)
# x_test1, y_test1 = dataset(x_test,y_test, y_test, TIME_STEPS)

x_train_vader, y_train_vader = dataset(x_train_vader,y_train_vader, y_train_vader, TIME_STEPS)
x_test_vader, y_test_vader = dataset(x_test_vader,y_test_vader, y_test_vader, TIME_STEPS)

x_train_exp, y_train_exp = dataset(x_train_exp,y_train_exp, y_train_exp, TIME_STEPS)
x_test_exp, y_test_exp = dataset(x_test_exp,y_test_exp, y_test_exp, TIME_STEPS)

In [21]:
# print(x_train1.shape, y_train1.shape)
# print(x_test1.shape, y_test1.shape)

# x_train1=x_train1.reshape((x_train1.shape[0], 1,x_train1.shape[1]))
# x_test1=x_test1.reshape((x_test1.shape[0], 1,x_test1.shape[1]))

# print(x_train1.shape, y_train1.shape)
# print(x_test1.shape, y_test1.shape)

x_train_vader=x_train_vader.reshape((x_train_vader.shape[0], 1,x_train_vader.shape[1]))
x_test_vader=x_test_vader.reshape((x_test_vader.shape[0], 1,x_test_vader.shape[1]))

x_train_exp=x_train_exp.reshape((x_train_exp.shape[0], 1,x_train_exp.shape[1]))
x_test_exp=x_test_exp.reshape((x_test_exp.shape[0], 1,x_test_exp.shape[1]))

In the below cell we will evaluate the performance of the data including the sentiment scores from VADER and Expertise. 

In [22]:
#Evaluation using the VADER
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_vader.shape[1],x_train_vader.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_vader,y_train_vader, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

CPU times: user 40 s, sys: 2.81 s, total: 42.8 s
Wall time: 29.7 s


In [23]:
time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.974926710128784
Base MSE for the above model: 0.0030550574883818626


In [24]:
#now evaluating the performance of the model using the Expert generated sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_exp.shape[1],x_train_exp.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_exp,y_train_exp, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

CPU times: user 39.3 s, sys: 2.61 s, total: 41.9 s
Wall time: 28.4 s


In [25]:
time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.07194471359253
Base MSE for the above model: 0.0023978634271770716


# Comparison of MSE for VADER and Expertise Sentiment

In [31]:
d = {'Vader Sentiment Analysis MSE': [0.00305], 'Expertise Sentiment analysis MSE': [0.00239]}
df = pd.DataFrame(data=d)
df

,Vader Sentiment Analysis MSE,Expertise Sentiment analysis MSE
0,0.00305,0.00239


Based on the above result we can conclude that expertised sentiment are providing lower MSE. Therefore, we will consider Expertise Sentiments for the further analysis.